In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
warnings.filterwarnings('ignore')


C:\Users\tatsuya.okamoto\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\tatsuya.okamoto\AppData\Local\Continuum\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


基本変数定義

In [2]:
submit_flg=1 #全件処理かどうかをコントロール
grid_flg=0 #GridSearchをするかコントロール

SEED=12345
sample_num=10000
fold_num=10

#train関連
train_dir='./feature/101_train.csv'
train_drop_col=['ID_code', 'target']
train_label='target'

#test関連
test_dir='./feature/101_test.csv'
test_drop_col=['ID_code']

#結果ファイル関連
oof_preds_dir='./oof/104_LightGBM_preds.csv'
test_preds_dir='./oof/104_LightGBM_tests.csv'
submission_dir='./submission/104_LightGBM_submission.csv'
save_col_name='oof_xgb'

sample_submission_dir='./submission/sample_submission.csv'
submission_target_col_name='target'
submission_id_col_name='ID_code'

In [3]:
#model_param
param = {
        'num_leaves': 6,
        'max_bin': 63,
        'min_data_in_leaf': 45,
        'learning_rate': 0.01,
        'min_sum_hessian_in_leaf': 0.000446,
        'bagging_fraction': 0.55, 
        'bagging_freq': 5, 
        'max_depth': 14,
        'save_binary': True,
        'seed': 31452,
        'feature_fraction_seed': 31415,
        'feature_fraction': 0.51,
        'bagging_seed': 31415,
        'drop_seed': 31415,
        'data_random_seed': 31415,
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbose': 1,
        'metric': 'auc',
        'is_unbalance': True,
        'boost_from_average': False,
    }

テンプレ処理

In [4]:
#ロード
train_df=pd.read_csv(train_dir)
test_df=pd.read_csv(test_dir)

In [5]:
#サンプリング
if not(submit_flg ==1):
    train_df=train_df.sample(n=sample_num,random_state=SEED)
    test_df=test_df.sample(n=sample_num,random_state=SEED)

In [6]:
#x,y作成
x_train=train_df.drop(train_drop_col,axis=1)
y_train=train_df[train_label]
x_test=test_df.drop(test_drop_col,axis=1)

GridSearch

In [7]:
if grid_flg ==1:
    clf = xgb.XGBRegressor()
    parameters = {
        'learning_rate':[0.1,0.01],
        'n_estimators':[10,100,1000],
        'max_depth':[3,4,5,10,15],
        'min_child_weight':[1,2,3],
        'max_delta_step':[0,5],
        'gamma':[0,3,10,30],
        'subsample':[0.8,1],
        'colsample_bytree':[0.8,1],
        'objective':['binary:logistic','reg:linear'],
        'booster':['gbtree','gblinear'],
        'nthread':[None,4],
        'scale_pos_weight':[1],
        'seed':[SEED]
    }
    clf = RandomizedSearchCV( estimator=model,
                                        param_distributions=parameters,
                                        cv=5,              #CV
                                        scoring="accuracy", #metrics
                                        n_jobs=1,           #num of core
                                        verbose=0,          
                                        random_state=1)
    clf.fit(x_train, y_train)

モデル実行

In [8]:
folds = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=SEED)
oof_preds = np.zeros((len(x_train), 1))
test_preds = np.zeros((len(x_test), 1))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(x_train.values, y_train.values)):
    print("\n")
    print("Fold {}".format(fold_))
    trn_x,trn_y = x_train.iloc[trn_idx], y_train.iloc[trn_idx]
    val_x,val_y = x_train.iloc[val_idx], y_train.iloc[val_idx]
    trn_data = lgb.Dataset(trn_x,trn_y)
    val_data = lgb.Dataset(val_x,val_y)
    
    num_round = 15000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 250)
    val_pred = clf.predict(val_x, num_iteration=clf.best_iteration)
    test_pred = clf.predict(x_test, num_iteration=clf.best_iteration)
    
    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    #print("val = {}".format(val_pred)
    oof_preds[val_idx, :] = val_pred.reshape((-1, 1))
    test_preds += test_pred.reshape((-1, 1))
    
test_preds /= fold_num
roc_score = roc_auc_score(y_train, oof_preds.ravel())
print("Overall AUC = {}".format(roc_score))



Fold 0
Training until validation scores don't improve for 250 rounds.
[1000]	training's auc: 0.862234	valid_1's auc: 0.846295
[2000]	training's auc: 0.889724	valid_1's auc: 0.871047
[3000]	training's auc: 0.902381	valid_1's auc: 0.88213
[4000]	training's auc: 0.910379	valid_1's auc: 0.888456
[5000]	training's auc: 0.915653	valid_1's auc: 0.891442
[6000]	training's auc: 0.919528	valid_1's auc: 0.893551
[7000]	training's auc: 0.922627	valid_1's auc: 0.894491
[8000]	training's auc: 0.92537	valid_1's auc: 0.895198
[9000]	training's auc: 0.928073	valid_1's auc: 0.895658
Early stopping, best iteration is:
[8843]	training's auc: 0.927643	valid_1's auc: 0.895715
AUC = 0.8957148557695099


Fold 1
Training until validation scores don't improve for 250 rounds.
[1000]	training's auc: 0.860613	valid_1's auc: 0.846678
[2000]	training's auc: 0.889101	valid_1's auc: 0.873472
[3000]	training's auc: 0.902565	valid_1's auc: 0.884724
[4000]	training's auc: 0.910306	valid_1's auc: 0.890473
[5000]	trainin

In [9]:
#結果保存
if submit_flg ==1:
    series_oof_preds = pd.Series(data=oof_preds[:,0], name=save_col_name, dtype='float')
    series_test_preds = pd.Series(data=test_preds[:,0], name=save_col_name, dtype='float')

    series_oof_preds.to_csv(oof_preds_dir,header=True, index=False)
    series_test_preds.to_csv(test_preds_dir,header=True, index=False)

    sample = pd.read_csv(sample_submission_dir)
    sample.target = test_preds[:,0].astype(float)
    sample.ID_code = test_df['ID_code']
    sample.to_csv(submission_dir, index=False)
    